In [1]:
import math
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import scipy.stats

### Functions

In [2]:
def Lnew():
    lnew=np.array([[1,1,1],
                   [1,2,2],
                   [1,3,3],
                   [2,1,2],
                   [2,2,3],
                   [2,3,1],
                   [3,1,3],
                   [3,2,1],
                   [3,3,2]])
    return(lnew)

In [3]:
def SNR1(Y):
    #eta
    #surface defects
    s=0
    I,J=np.shape(Y)
    s=0
    for i in range(I):
        for j in range(J):
            s+=T[i][j]
    return(-10*math.log(s,10))

In [4]:
def SNR2(T):
    #eta'
    #thickness
    I,J=np.shape(T)
    s=0
    for i in range(I):
        for j in range(J):
            s+=T[i][j]
    mu=(1/(I*J))*s
    mu2=(mu)**2
    sig2=0
    for i in range(I):
        for j in range(J):
            s+=(T[i][j]-mu)**2
    sig2=(1/(I*J-1))*sig2
    return(10*math.log(mu2/sig2,10))

In [5]:
def SNR3(R):
    #eta"
    #deposition rate
    res=[]
    for i in R:
        res.append(20*math.log(i,10))
    return(res)

In [6]:
def effect_index(A,F):
    res=[]
    for i in range(len(F)):
        if F[i]!=0:
            #D=[]
            for j in set(A[:,i]):
                x=[F[i]+str(j),np.where(A[:,i]==j)]
                res.append(x)
            #res.append(D)
    return(res)

### Thickness Uniformity

In [7]:
F=['A','B','C'] #input manually
A=Lnew()
n2=[46,44,42,45,41,37,42,39,33] #input manually
E=effect_index(A,F)
m_list=[]
for i in range(len(E)):
    n=0
    for j in E[i][1][0]:
        n+=n2[j]
    n/=len(E[i][1][0])
    m_list.append((E[i][0],n))
m_list

[('A1', 44.0),
 ('A2', 41.0),
 ('A3', 38.0),
 ('B1', 44.333333333333336),
 ('B2', 41.333333333333336),
 ('B3', 37.333333333333336),
 ('C1', 40.666666666666664),
 ('C2', 40.666666666666664),
 ('C3', 41.666666666666664)]

In [8]:
m=np.mean(n2)
m

41.0

In [9]:
SS_gtot=0
for i in n2:
    SS_gtot+=i**2
SS_gtot

15265

In [10]:
SS_mean=18*(m**2)
SS_mean

30258.0

In [11]:
SS_tot=0
for i in n2:
    SS_tot+=(i-m)**2
SS_tot

136.0

In [12]:
SS_effect={}
for i in range(len(F)):
    if F[i]!=0:
        SS_effect[F[i]]=0
        for j in range(len(m_list)):
            if m_list[j][0].startswith(F[i]):
                SS_effect[F[i]]+=(m_list[j][1]-m)**2
for j in SS_effect.keys():
    SS_effect[j]*=3
SS_effect

{'A': 54.0, 'B': 74.0, 'C': 2.0}

In [13]:
SS_error=SS_tot-sum(SS_effect.values())
SS_error

6.0

In [14]:
DOF=[2,2,2] #input manually
DOF.append(len(DOF)+2)
DOF.append(sum(DOF))
SS=[SS_effect[i] for i in SS_effect.keys()]+[SS_error,SS_tot]
SS

[54.0, 74.0, 2.0, 6.0, 136.0]

In [15]:
DOF

[2, 2, 2, 5, 11]

In [16]:
MS=[SS[i]/DOF[i] for i in range(len(SS))]
MS

[27.0, 37.0, 1.0, 1.2, 12.363636363636363]

In [17]:
error_index=[-3] + [-2] #input the 1st list manually
Error_SS=sum([SS[i] for i in error_index])
Error_MS=Error_SS/sum([DOF[i] for i in error_index])
F_ratio=[i/Error_MS for i in MS[:-1]]
F_ratio

[23.625, 32.375, 0.875, 1.05]

In [18]:
Error_MS

1.1428571428571428

In [19]:
Error_SS

8.0

In [20]:
sig2_exp=Error_MS
sig_exp=np.sqrt(sig2_exp)
sig2_effect=sig2_exp*(1/3) 
sig_effect=np.sqrt(sig2_effect)
print("sig_exp = {}".format(sig_exp))
print("sig_effect = {}".format(sig_effect))

sig_exp = 1.0690449676496976
sig_effect = 0.6172133998483676


In [21]:
current_cond=["A1","B2","C2","D2","E3"]#input manually
for i in m_list:
    if i[0] in current_cond:
        print(i[0][0]+"-limits: "+str([i[1]-2*sig_effect,i[1]+2*sig_effect]))

A-limits: [42.765573200303265, 45.234426799696735]
B-limits: [40.0989065336366, 42.56776013303007]
C-limits: [39.43223986696993, 41.9010934663634]


In [22]:
n=len(n2)
#Optimum settings: A3,B1       #pick manually(significant terms outside effect limits)
nA3=6
nB1=6
n0=1/(1/n+(1/nA3-1/n)+(1/nB1-1/n)) #edit according to optimum settings chosen
print("Equivalent sample size: {}".format(n0))

Equivalent sample size: 4.5


In [23]:
sig2_pred=(1/n0)*(sig2_exp)
print("Prediction Error: {}".format(sig2_pred))

Prediction Error: 0.25396825396825395
